# 1. Import Libraries

In [1]:
import time
import multiprocessing
import pandas as pd
import numpy as np
import random
from numpy import inf

# 2. Parameters

In [1]:
max_iter = 10           # Number of Iterations
max_transaction = 10    # Number of Transactions per Ant
ant = 10                # Number of Ants
rho = 0.10              # Pheromone Evaporation Rate
tau = 0                 # Initial Pheromone
alpha = 2               #pheromone exponential parameter
beta = 1                #desirability exponential parameter

# 3. Data Sets

In [3]:
dataOL = pd.read_excel(r'sclp.xlsx', sheet_name='OrderList')
dataFR = pd.read_excel(r'sclp.xlsx', sheet_name='FreightRates')
dataWhCost = pd.read_excel(r'sclp.xlsx', sheet_name='WhCosts')
dataWhCap = pd.read_excel(r'sclp.xlsx', sheet_name='WhCapacities')

# 4. Data Preprocessing

In [4]:
#1 ORDER LIST

# 1.1 There is only one date in the order list, threfore it wont play any significant role for the optimization.
date_check = dataOL.groupby(['Order Date']).size()
print(date_check) 

# 1.2 Order Date, TPT, Ship ahead day count, Ship Late Day count will be removed as they are not within the scope of this optimization.
dataOL=dataOL.drop(columns=['Order Date','TPT','Ship ahead day count','Ship Late Day count'])

# 1.3 carrier_check is to check the distinct value of carriers. Which then will be used to filter only the necessary carriers for the transportation.
carrier_check = dataOL.groupby(['Carrier']).size().reset_index()
carrier_list = pd.Series({cc: carrier_check[cc].unique() for cc in carrier_check})

# 1.4 port_check is to check the distinct value of ports. Which then will be used to filter only the necessary origin ports for the transportation.
port_check = dataOL.groupby(['Origin Port']).size().reset_index()
port_list = pd.Series({pc: port_check[pc].unique() for pc in port_check})

# 1.5 plant_check is to check the distinct value of warehouses. Which then will be used to filter only the necessary warehouses for the transportation.
#     It will also be used to filter the warehouse needed to use based on the Warehouse Cost Dataframe and also Warehouse Capacity DataFrame.
plant_check = dataOL.groupby(['Plant Code']).size().reset_index()
plant_list = pd.Series({pc: plant_check[pc].unique() for pc in plant_check})

# [2 FREIGHT RATE]
# 2.1 Filter the Freight Rate DataFrame to only get the necessary carriers to the existing Order List DataFrame.
dataFR = dataFR[dataFR['Carrier'].isin(carrier_list['Carrier'])]

# 2.2 Filter the Freight Rate DataFrame to only get the necessary carriers to the existing Order List DataFrame.
dataFR = dataFR[dataFR['orig_port_cd'].isin(port_list['Origin Port'])]

# 2.3 Drop out tpt_day_cnt and carrier type as they are not needed for the optimization.
dataFR = dataFR.drop(columns=['tpt_day_cnt','Carrier type'])

# 3 [WAREHOUSE COST]
# 3.1 Filter the Warehouse Cost DataFrame to only get the necessary warehouses to the existing Order List DataFrame.
dataWhCost = dataWhCost[dataWhCost['WH'].isin(plant_list['Plant Code'])]

# 4 [WAREHOUSE CAPACITY]
# 4.1 Filter the Warehouse Cost DataFrame to only get the necessary warehouses to the existing Order List DataFrame.
dataWhCap = dataWhCap[dataWhCap['Plant ID'].isin(plant_list['Plant Code'])]


Order Date
2013-05-26    9215
dtype: int64


In [57]:
x = [184.0,185.0]
print(dataFR.loc[x])

      Carrier orig_port_cd dest_port_cd  minm_wgh_qty  max_wgh_qty svc_cd  \
184.0  V444_0       PORT04       PORT09         500.0      1999.99    DTD   
185.0  V444_0       PORT04       PORT09           0.0        99.99    DTD   

       minimum cost    rate mode_dsc  Pheromone  
184.0        3.4552  0.0544   AIR      0.000000  
185.0        3.4552  0.0824   AIR      0.029235  


# 5. Main Function

In [13]:
#Pheromone Initialisation
dataOL['Pheromone'] = tau
dataFR['Pheromone'] = tau
prev_ant_history = pd.DataFrame()

#Cost Initialisation
lowest_cost_index = 0
lowest_cost = 0

#The number of Iterations of the Ant Colony.
for colony_iter in range(max_iter):

    #The number of Ants in a single Colony Iteration.
    processes= []
    res = []
    
    res_aco_t = multiprocessing.Queue()

    for _ in range(ant):

        p = multiprocessing.Process(target=aco_traversal, args=(colony_iter,max_transaction,res_aco_t,prev_ant_history,dataOL, dataFR, dataWhCap, dataWhCost))
        p.start()
        processes.append(p)
        
    for process in processes:
        process.join()

    for _ in range(ant):
        x = res_aco_t.get()
        res.append(x)

    prev_ant_history = pd.DataFrame()
    for prev in range(len(res)):
        prev_ant_history = pd.concat([prev_ant_history,res[prev][0]], ignore_index = True, axis = 0)

    fitness = objective_function(res,lowest_cost,lowest_cost_index,colony_iter,alpha,beta)

    lowest_cost_index = fitness[1]
    lowest_cost = fitness[0]

    dataOL,dataFR = pheromone_update(res,fitness,rho)
    # print(dataOL.where(dataOL['Pheromone']>0).dropna())
    print(prev_ant_history)

[ANT ENTERING]
[ANT ENTERING][ANT ENTERING]
[ANT ENTERING]

[ANT ENTERING]
[ANT ENTERING]
[ANT ENTERING]
[ANT ENTERING]
[ANT ENTERING]
[ANT ENTERING]
[ANT DONE]
[ANT DONE]
[ANT DONE]
[ANT DONE]
[ANT DONE]
[ANT DONE]
[ANT DONE]
[ANT DONE]
[ANT DONE]
[ANT DONE]
[  1  ] - LOWEST COST: $ 105434.04490218288

Fitness
[0.8830175815065386, 1.0, 0.2061266138538316, 0.47224331751431514, 0.39630669928918794, 0.14376504731125103, 0.45713692416328267, 0.12264095592355852, 0.2914738999151171, 0.38604279313045003]
      Port Plant Code      Order ID Service ID     Carrier Carrier Index  \
0   PORT04    PLANT03  1.447391e+09        CRF  No Carrier    No Carrier   
1   PORT04    PLANT03  1.447423e+09        DTP      V444_0        1519.0   
2   PORT09    PLANT16  1.447202e+09        CRF  No Carrier    No Carrier   
3   PORT04    PLANT03  1.447393e+09        CRF  No Carrier    No Carrier   
4   PORT09    PLANT16  1.447398e+09        CRF  No Carrier    No Carrier   
..     ...        ...           ...    

# 6. Funtions

## 6.1 ACO Traversal Function

In [12]:
def aco_traversal(colony_iter,max_transaction,res_aco_t,prev_ant_transactions,dataOL, dataFR, dataWhCap, dataWhCost):
    print('[ANT ENTERING]')
    current_transaction = 0 # The current Order Transaction for an Ant.
    result = []
    total_cost = 0
    dataOL_sub, dataFR_sub, dataWhCap_sub, dataWhCost_sub = fresh_dataframes(dataOL, dataFR, dataWhCap, dataWhCost)

    dataOL_sub['Status'] = True
    dataFR_sub['Status'] = True
    current_record = pd.DataFrame()
    order_carrier_record = pd.DataFrame()
    prev_ant = prev_ant_transactions
    sel_prev = []
    
    #The Loop Will Go On Until Max Number of Transactions Has Been Reached.
    while current_transaction < max_transaction:
        # print(current_transaction)
        route_status = True
        use_carrier = True
        valid = True                      #Does the logistics requires carriers?
        order_carrier_status = False            #Has the carrier been used?
        carrier_index,carrier_index_index = 0,0 # Index for The Carrier and The Index's Index

        f_data_ol_status = (dataOL_sub['Status'] == True) # [FILTER]

        # ORDER ID SELECTION SEGMENT
        order_id_availability = dataOL_sub.where(f_data_ol_status).groupby(['Order ID','Origin Port','Plant Code','Service Level','Carrier','Status'])['Pheromone'].sum().reset_index()
        selected_order_id = selection(order_id_availability,'[ORDER ID SELECTION]')

       # CARRIER SELECTION SEGMENT
        if colony_iter == 0:
            if selected_order_id['Carrier'][0] != 'V44_3':
                f_carrier_status = (dataFR_sub['Status'] == True)
                f_carrier_sel = (dataFR_sub['Carrier'] == selected_order_id['Carrier'][0])
                carrier_availability = dataFR_sub.where(f_carrier_sel & f_carrier_status).groupby([dataFR_sub.index,'Carrier','Status'])['Pheromone'].sum().reset_index()
                selected_carrier = selection(carrier_availability,'[CARRIER SELECTION]')

            else:
                use_carrier = False
            
        else:
            if selected_order_id['Carrier'][0] != 'V44_3':
                f_prev_ant_status = (prev_ant['Status'] == True)
                f_carrier_status = (dataFR_sub['Status'] == True)
                f_sel_order_index = (prev_ant['Order ID'] == selected_order_id['Order ID'][0])
                sel_prev = prev_ant.where(f_prev_ant_status & f_sel_order_index).dropna()
                sel_prev = sel_prev['Carrier Index'].tolist()
                dataFR_sub_ = dataFR_sub.loc[sel_prev]
                carrier_availability = dataFR_sub_.groupby([dataFR_sub_.index,'Carrier','Status'])['Pheromone'].sum().reset_index()
                selected_carrier = selection(carrier_availability,'[CARRIER SELECTION]')

                if selected_carrier.empty is True:
                    route_status = False
            else:
                use_carrier = False

        # COST CALCULATION
        if route_status:
            if use_carrier:
                t_cost = transportation_cost(selected_order_id,selected_carrier,dataOL_sub,dataFR_sub)

                if type(t_cost) is str:
                    route_status = False
                else:
                    total_cost += t_cost
            
            wh_cost = warehouse_cost(dataWhCost_sub,dataWhCap_sub,selected_order_id,selected_order_id['Plant Code'][0],dataOL_sub)
            
            if type(wh_cost) is str:
                route_status = False
            else:
                total_cost += wh_cost

        #AVAILABILITY UPDATE
        if route_status:

            # UPDATE FOR ORDER ID
            f_data_OL_index = dataOL_sub[dataOL_sub['Order ID']==selected_order_id['Order ID'][0]].index.values #filter index finding
            dataOL_sub.at[f_data_OL_index,'Status'] = False

            #UPDATE FOR WAREHOUSE CAP
            f_wh_cap_index = dataWhCap_sub[dataWhCap_sub['Plant ID']==selected_order_id['Plant Code'][0]].index.values #filter index finding
            counter = dataWhCap_sub['Daily Capacity '][f_wh_cap_index]-1
            dataWhCap.at[f_wh_cap_index,'Daily Capacity '] = counter

            #UPDATE NUMBERS OF TRANSACTIONS
            current_transaction+=1
            # print(current_transaction)
            
            #UPDATE FOR CARRIER
            if use_carrier:
                # UPDATE FOR FRIEGHT RATE
                dataFR_sub.at[selected_carrier['level_0'][0],'Status'] = False

                if colony_iter > 0:
                    f_prev_avail = prev_ant[prev_ant['Carrier Index']==selected_carrier['level_0'][0]].index.values
                    f_prev_avail2 = prev_ant[prev_ant['Order ID']==selected_order_id['Order ID'][0]].index.values
                    prev_ant.at[f_prev_avail,'Status'] = False
                    prev_ant.at[f_prev_avail2,'Status'] = False

            #UPDATE LOGISTIC RECORD FOR Nth ANT
            if use_carrier:
                current_record = pd.DataFrame( {'Port': [selected_order_id['Origin Port'][0]],
                                                'Plant Code' : [selected_order_id['Plant Code'][0]],
                                                'Order ID' : [selected_order_id['Order ID'][0]],
                                                'Service ID' : [selected_order_id['Service Level'][0]],
                                                'Carrier' : [selected_carrier['Carrier'][0]],
                                                'Carrier Index' : [selected_carrier['level_0'][0]],
                                                'Status' : [True],
                                                'Pheromone' : 0,
                                                'Cost' : total_cost})
            else:
                current_record = pd.DataFrame( {'Port': [selected_order_id['Origin Port'][0]],
                                                'Plant Code' : [selected_order_id['Plant Code'][0]],
                                                'Order ID' : [selected_order_id['Order ID'][0]],
                                                'Service ID' : [selected_order_id['Service Level'][0]],
                                                'Carrier' : 'No Carrier',
                                                'Carrier Index' : 'No Carrier',
                                                'Status' : [True],
                                                'Pheromone' : 0,
                                                'Cost' : total_cost})

            order_carrier_record = pd.concat([order_carrier_record,current_record], ignore_index = True, axis = 0)


    result = [order_carrier_record]
    res_aco_t.put(result)
    print('[ANT DONE]')

## 6.2 Selection Function

In [6]:
def selection(dat,type_sel):

    cumulative_pheromone,cum_p_segment = 0,0    # Total Pheromones and Each Segmentation of Pheromone.
    start_range,end_range = 0,0                 # Range of Each Segmented Pheromone.
    data_length = 0                             # Length of Data to be segmentized.
    selection_list = []                         # Selection of choices to be segmentized.
    roulette_value = random.uniform(0, 1)       # Roulette Value.
    target = 0                                  # The Target answer after the Roullete Wheel.

    # 1. Eliminate Routes With False Status.
    f_status_count = dat['Status'] == True          # [FILTER] Filter the status to only True.
    dat = dat.where(f_status_count).dropna()        # Filter the data status to only True and drop NaN values.
    data_length = len(dat.index)                    # Find the length of data.
    cumulative_pheromone = dat['Pheromone'].sum()   # Accumulate Pheromone for Selection.
    
    #2. Normalize the Routes in a Uniform Roulette Wheel.
    for index,row in dat.iterrows():
        cur_list=[]

        # 2.1 First ACO Iteration or specfically when the Accumulated Pheromone is 0.
        if cumulative_pheromone == 0:
            end_range = (index+1)/data_length

        # 2.2 next Iterations.
        else:
            end_range = (cum_p_segment+row['Pheromone']/cumulative_pheromone)
            cum_p_segment += row['Pheromone']

        # 2.3 Segmentize by range of index/pheromone.
        cur_list = [row[0],start_range,end_range]
        selection_list.append(cur_list)
        start_range = end_range

    # 3. Applying the Roulette Wheel to get the Target answer. 
    for sel_ in selection_list:
        if sel_[1] <= roulette_value and sel_[2] >= roulette_value:
            probability = "{:.2f}".format((sel_[2]-sel_[1])*100)
            target = sel_[0]

    # 4. If the Data is based on Order ID.
    if dat.columns[0] == 'Order ID':
        f_order = dat['Order ID'] == target
        target = dat.where(f_order).groupby(['Order ID','Origin Port','Plant Code','Service Level','Carrier']).sum().reset_index()
        target['Size'] = data_length
    
    # 5 If the Data is based on Carrier.
    elif dat.columns[1] == 'Carrier':
        f_carrier = dat['level_0'] == target
        target = dat.where(f_carrier).groupby(['level_0','Carrier']).sum().reset_index()
        target['Size'] = data_length

    # print(type_sel,'\tSelected Target is ',target,' with the Probability of ',probability,'% [ Roulette Value : ',roulette_value," ]")
    return target

## 6.3 Objective/Fitness Function

In [7]:
def objective_function(res,lowest_cost,lowest_cost_index,iter,alpha):

    if iter == 0:
        lowest_cost = res[0][0]['Cost'].sum()
        lowest_cost_index = 0
        
    fitness = []

    for index_ocr,ocr in enumerate(res):
        if ocr[0]['Cost'].sum() < lowest_cost:
            lowest_cost = ocr[0]['Cost'].sum()
            lowest_cost_index = index_ocr

    for index_ocr,ocr in enumerate(res):
        fitness_value = 1/(ocr[0]['Cost'].sum()/lowest_cost)
        fitness.append(fitness_value)
    
    fitness[2] = [fit ** alpha for fit in fitness[2]]


    print('[ ',lowest_cost_index,' ] - LOWEST COST: $',lowest_cost)
    print('\nFitness')
    print(fitness)
    
    return lowest_cost,lowest_cost_index, fitness

## 6.4 Transportation Cost Function

In [8]:
def transportation_cost(selected_order_id,selected_carrier,dataOL_sub,dataFR_sub):
    
    transportation_cost = 0
    data_OL_tc = dataOL_sub.loc[dataOL['Order ID'] == selected_order_id['Order ID'][0]].reset_index()
    data_FR_tc = dataFR_sub.loc[selected_carrier['level_0'][0]]

    if data_OL_tc['Weight'][0] < data_FR_tc['minm_wgh_qty']:
        transportation_cost = 'Not Enough Weight'
    elif data_OL_tc['Weight'][0] > data_FR_tc['max_wgh_qty']:
        transportation_cost = 'Exceeded Weight'
        
    else:
        if data_FR_tc['mode_dsc'] == 'GROUND':
            transportation_cost = (data_FR_tc['rate']/data_FR_tc['max_wgh_qty'])*data_OL_tc['Weight'][0]
        else:
            transportation_cost = data_FR_tc['rate']*data_OL_tc['Weight'][0]

        if transportation_cost < data_FR_tc['minimum cost']:
            transportation_cost = data_FR_tc['minimum cost']

    return transportation_cost

## 6.5 Warehouse Cost and Capacity Function

In [9]:
def warehouse_cost(dataWhCost_sub,dataWhCap_sub,selected_order_id,selected_plant_code,dataOL_sub):

    data_Cap = dataWhCap_sub.loc[dataWhCap_sub['Plant ID'] == selected_plant_code].reset_index()
    data_Cap_index = dataWhCap_sub.loc[dataWhCap_sub['Plant ID'] == selected_plant_code]
    data_Cap_index = data_Cap_index.index[0]
    data_Cost = dataWhCost_sub.loc[dataWhCost_sub['WH'] == selected_plant_code].reset_index()
    data_OL = dataOL_sub.loc[dataOL_sub['Order ID'] == selected_order_id['Order ID'][0]].reset_index()
    wh_cost = 0

    if data_Cap['Daily Capacity '][0] > 0:
        wh_cost = data_OL['Unit quantity'][0]*data_Cost['Cost/unit'][0]

    else:
        wh_cost = 'Warehouse is Full!'

    return wh_cost

## 6.6 Pheromone Update Function 

In [4]:
def pheromone_update(res,fitness,rho):

    #Pheromone Evaporation
    dataOL['Pheromone'] = dataOL['Pheromone']*(1-rho)
    dataFR['Pheromone'] = dataFR['Pheromone']*(1-rho)

    for ft in range(len(fitness[2])):

        #Pheromone Update
        carrier_l = res[ft][0]['Carrier Index'].where(res[ft][0]['Carrier Index'] != 'No Carrier').dropna().tolist()
        order_l = res[ft][0]['Order ID'].tolist()

        for ol in range(len(order_l)):
            f_data_OL_index = dataOL[dataOL['Order ID']==order_l[ol]].index.values
            dataOL.at[f_data_OL_index,'Pheromone'] = dataOL['Pheromone'][f_data_OL_index] + fitness[2][ft]

        if carrier_l:
            for cl in range(len(carrier_l)):
                dataFR.at[carrier_l[cl],'Pheromone'] = dataFR['Pheromone'][int(carrier_l[cl])] + fitness[2][ft]

    return dataOL,dataFR

## 6.7 Fresh DataFrames

In [11]:
def fresh_dataframes(OL, FR, WCap, WCost):

    dataOL, dataFR, dataWhCap, dataWhCost = OL, FR, WCap, WCost
    return dataOL, dataFR, dataWhCap, dataWhCost